### Avoids scroll-in-the-scroll in the entire Notebook

In [1]:
from IPython.display import Javascript
def resize_colab_cell():
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

# Functions for **`Embeddings`** and **`Cosine Similarity`** sores

### Function to get Embeddings from Tokenized array

In [2]:
!pip install transformers

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.3 MB/s eta 0:00:00


In [3]:
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


def get_embeddings(tokenized_array):
    tokenized_array_input = " ".join(tokenized_array)
    tokenized_array_encoding = tokenizer(tokenized_array_input, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        tokenized_array_embedding = model(**tokenized_array_encoding).last_hidden_state.mean(dim=1).numpy()

    return tokenized_array_embedding

<IPython.core.display.Javascript object>

### Function to get *`Cosine Similarity`*

In [4]:
def get_cosine_similarity(JD_req_skills_embeddings, JD_req_edu_embeddings, CV_skills_embeddings, CV_edu_embeddings):
    skills_score = cosine_similarity(JD_req_skills_embeddings, CV_skills_embeddings)
    edu_score = cosine_similarity(JD_req_edu_embeddings, CV_edu_embeddings)
    return (skills_score + edu_score) / 2.0

<IPython.core.display.Javascript object>

### Function to get Cosine Similarity for Positions

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidf_vectorizer = TfidfVectorizer()

<IPython.core.display.Javascript object>

In [6]:
def get_cosine_sim_for_category(position, category):

    tfidf_matrix = tfidf_vectorizer.fit_transform([position, category])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    return cosine_sim[0][0]

<IPython.core.display.Javascript object>

# Preprocessing **`Job Description`**

### Importing dataset from Hugging Face

In [7]:
!pip install datasets

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [8]:
from datasets import load_dataset

dataset = load_dataset("jacob-hugging-face/job-descriptions")
print(dataset)

<IPython.core.display.Javascript object>

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
        num_rows: 853
    })
})


### Extracting *`Company Name`*

In [9]:
def JD_extract_company(job_num):
    return dataset["train"][job_num]["company_name"]

<IPython.core.display.Javascript object>

### Extracting *`Position`*

In [10]:
def JD_extract_position(job_num) :
    return dataset["train"][job_num]["position_title"]

<IPython.core.display.Javascript object>

### Extracting *`Required Skills`*

In [11]:
def JD_extract_required_skills(job_num):
    model_response = eval(dataset["train"][job_num]["model_response"])
    return model_response["Required Skills"] if "Required Skills" in model_response else "N/A"

<IPython.core.display.Javascript object>

### Extracting *`Educational Requirements`*

In [12]:
def JD_extract_educational_requirements(job_num):
    model_response = eval(dataset["train"][job_num]["model_response"])
    return model_response["Educational Requirements"] if "Educational Requirements" in model_response else "N/A"

<IPython.core.display.Javascript object>

# Preprocessing **`Resume Data`**
> needs attention (to mount google drive & `.zip` file upload)

### Extracting text from resume PDF

In [13]:
!pip install PyPDF2

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [14]:
import PyPDF2


def extract_resume_text(filePath):
    reader = PyPDF2.PdfReader(open(filePath, "rb"))

    text = ""
    for i in range(len(reader.pages)): text += reader.pages[i].extract_text()
    return text

<IPython.core.display.Javascript object>

### Extracting *`Category (Job Role)`*

In [15]:
def extract_category(text):
     return text.strip().split("\n")[0].strip() if text.strip() else None

<IPython.core.display.Javascript object>

### Extracting *`Education`*

In [16]:
import re


def extract_education(text):
    education_keywords = ['Bsc', 'B. Pharmacy', 'B Pharmacy', 'Msc', 'M. Pharmacy', 'Ph.D', 'Bachelor', 'Master']
    education = []

    for word in education_keywords:
        pattern = r"(?i)\b{}\b".format(re.escape(word))
        match = re.search(pattern, text)
        if match: education.append(match.group())

    return education

<IPython.core.display.Javascript object>

### Extracting *`Skills`*

In [17]:
!pip install spacy
!python -m spacy info
!python -m pip freeze | grep spacy
!python -m spacy download en_core_web_sm

<IPython.core.display.Javascript object>

2023-09-16 19:07:18.965511: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.6.1                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.109+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.6.0)        

en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl#sha256=83276fc78a70045627144786b52e1f2728ad5e29e5e43916ec37ea9c26a11212
spacy==3.6.1
spacy-legacy==3.0.12
spacy-loggers==1.0.4
2023-09-16 19:07:37.129843: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the pack

In [18]:
import spacy
nlp = spacy.load("en_core_web_sm")


def extract_skills(text):
    skills = []
    doc = nlp(text)

    for word in doc:
        if "NN" in word.tag_: skills.append(word.text)

    return list(set(skills))

<IPython.core.display.Javascript object>

### Uploading Resume PDFs as `Zip` and Extracting
> needs attention (to upload `.zip` file)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

<IPython.core.display.Javascript object>

Mounted at /content/drive


In [21]:
import zipfile, os


# location/path of zip file in colab runtime storage
resume_folder = "/content/drive/MyDrive/Resume-to-Job-Matcher/dataset/resume_PDF_dataset.zip"

if zipfile.is_zipfile(resume_folder):
    with zipfile.ZipFile(resume_folder, 'r') as zip_ref:
        zip_ref.extractall()
        path = os.getcwd() + "/" + os.path.splitext(os.path.basename(resume_folder))[0]
else:
    print("# ERROR: Please upload a valid ZIP file with resume PDFs.")

<IPython.core.display.Javascript object>

# Getting Embeddings for JD & CV
> processing time may differ based on the dataset size

### For progress bar

In [22]:
!pip install tqdm
from tqdm import tqdm

<IPython.core.display.Javascript object>

### For *`Job Description`*

In [23]:
JD_req_skills_embeddings_list = []
JD_req_edu_embeddings_list = []
JD_position_list = []

jobs = 10
for job_num in tqdm(range(jobs), desc="Processing Jobs"):
    JD_req_skills_embeddings_list.append(get_embeddings(JD_extract_required_skills(job_num)))
    JD_req_edu_embeddings_list.append(get_embeddings(JD_extract_educational_requirements(job_num)))
    JD_position_list.append(JD_extract_position(job_num))

<IPython.core.display.Javascript object>

Processing Jobs: 100%|██████████| 10/10 [00:07<00:00,  1.30it/s]


### For *`Resume/CVs`*

In [25]:
import glob


CV_skills_embeddings_list = []
CV_education_embeddings_list = []
CV_category_list = []

pdf_files = glob.glob(os.path.join(path, "*.pdf"))

# for cv in pdf_files:
for cv in tqdm(pdf_files, desc="Processing resume/CVs"):
    thisPDF_text = extract_resume_text(cv)

    CV_skills_embeddings_list.append(get_embeddings(extract_skills(thisPDF_text)))
    CV_education_embeddings_list.append(get_embeddings(extract_education(thisPDF_text)))
    CV_category_list.append(extract_category(thisPDF_text))

<IPython.core.display.Javascript object>

Processing resume/CVs: 100%|██████████| 2484/2484 [49:27<00:00,  1.19s/it]


# Driver Function
> processing time may differ based on the dataset size

In [132]:
import PyPDF2, glob, os


def main():
    # outer loop (jobs)
    for job_num in tqdm(range(jobs), desc="Progress"):
        CV_and_score = []

        # inner loop (resume/CVs)
        for cv_idx, cv in enumerate(pdf_files):
            score = (get_cosine_similarity(JD_req_skills_embeddings_list[job_num], JD_req_edu_embeddings_list[job_num], CV_skills_embeddings_list[cv_idx], CV_education_embeddings_list[cv_idx]) + get_cosine_sim_for_category(JD_position_list[job_num], CV_category_list[cv_idx])) / 2.0
            CV_and_score.append((score[0][0], os.path.basename(cv)))
        # inner loop end

        CV_and_score.sort(key=lambda x: x[0], reverse=True)
        top_CVs = [filename for _, filename in CV_and_score[:5]]
        top_scores = [score for score, _ in CV_and_score[:5]]

        print(f"\nCompany: {JD_extract_company(job_num)} ({JD_position_list[job_num]}): \nTop 5 CVs: {top_CVs} \nCorresponding Scores: {top_scores}\n")
    # outer loop end


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [133]:
if __name__ == "__main__": main()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Progress:  10%|█         | 1/10 [00:06<01:01,  6.78s/it]

...


Company: Zander Insurance Agency (Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '37058472.pdf', '68240723.pdf', '13014900.pdf'] 
Corresponding Scores: [0.79591775, 0.71779966, 0.59922385, 0.5852879, 0.5825515]



Progress:  90%|█████████ | 9/10 [01:16<00:08,  8.52s/it]


Company: Tuff (Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '37058472.pdf', '68240723.pdf', '13014900.pdf'] 
Corresponding Scores: [0.7972343, 0.71877515, 0.60078895, 0.58678734, 0.58364415]



Progress: 100%|██████████| 10/10 [01:24<00:00,  8.42s/it]


Company: General Dynamics Information Technology (SR. Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '37058472.pdf', '39776400.pdf', '34051710.pdf'] 
Corresponding Scores: [0.60813713, 0.50758064, 0.4418987, 0.43063515, 0.42894423]

